In [2]:
import pandas as pd
import numpy as np

## データ値に対する処理
- 欠測値(NaN)
- 異常値
- 単純重複データ
- 同じ意味、異なる名称の重複データ
- 重複属性(多重共線性)
- 不規則なデータ収集(間隔,単位)

In [3]:
sample = pd.read_csv('data/csv_exam_nan.csv')
sample

,math,english,science
0,70.0,NaN,NaN
1,75.0,65.0,80.0
2,NaN,NaN,NaN
3,56.0,89.0,NaN
4,89.0,95.0,83.0
5,90.0,100.0,89.0


# 欠測値処理 - 削除
- 欠測値が一つでもあるレコードを削除を削除
- すべての値が欠測のレコードを削除
- 欠測値が一つでもあるデータだけを選択

In [4]:
#「欠測値(NaN)」が一つでもあるレコードを削除
# df.dropna(how='any'(default)、 inplace=True)
# inplaceパラメータ:原本変数にすぐ適用する。
sample.dropna()

,math,english,science
1,75.0,65.0,80.0
4,89.0,95.0,83.0
5,90.0,100.0,89.0


In [5]:
sample #inplace処理をしていないため、原本に反映されない。

,math,english,science
0,70.0,NaN,NaN
1,75.0,65.0,80.0
2,NaN,NaN,NaN
3,56.0,89.0,NaN
4,89.0,95.0,83.0
5,90.0,100.0,89.0


In [6]:
# すべての値が欠測値であるRowだけを削除。
# df.dropna(how='all')
sample.dropna(how='all')

,math,english,science
0,70.0,NaN,NaN
1,75.0,65.0,80.0
3,56.0,89.0,NaN
4,89.0,95.0,83.0
5,90.0,100.0,89.0


In [7]:
# 決測値が一つでもあるデータだけを選択
# 条件索引
# df[df.isnull()].any(axis=1)]
sample[sample.isnull().any(axis=1)]

,math,english,science
0,70.0,NaN,NaN
2,NaN,NaN,NaN
3,56.0,89.0,NaN


In [8]:
# 結果値ベースでColumnに対して欠測値の有無をbool typeで返却
# df.isnull().any()
sample.isnull().any()

math       True
english    True
science    True
dtype: bool

In [9]:
sample.isnull().sum()

math       1
english    2
science    3
dtype: int64

## 欠測値処理 - 代替値
- 連続型:任意の値段(0)、mean、median、予測値、Domain知識活用
- 名目型 : mode(最頻値)、予測値、ドメイン式活用

In [10]:
# 連続型 - 任意の値を代替
# df.fillna()
sample.fillna(0)

,math,english,science
0,70.0,0.0,0.0
1,75.0,65.0,80.0
2,0.0,0.0,0.0
3,56.0,89.0,0.0
4,89.0,95.0,83.0
5,90.0,100.0,89.0


In [11]:
# mean-1)全体データの平均値
# sample:6 x 3のすべてのデータに対する平均
# df.mean() : Column別の平均値
sample.mean()

math       76.00
english    87.25
science    84.00
dtype: float64

In [12]:
# 平均を「決測値」に代入すること
# df.values.mean() : arrayタイプの演算でNaN値が一つでもあれば最終結果はNaN
# sample.values.mean() -> NaN
# df.fillna(0)->mean()
tot_avg = sample.fillna(0).values.mean()
sample.fillna(tot_avg)

,math,english,science
0,70.0,54.5,54.5
1,75.0,65.0,80.0
2,54.5,54.5,54.5
3,56.0,89.0,54.5
4,89.0,95.0,83.0
5,90.0,100.0,89.0


In [13]:
# mean-2)ギョルチュクチが存在する属性(変数=コラム)の平均値
# sample : math/en/scのColumnに存在する欠測値 -> math/en/sc Columnの平均値
# df.mean() : Column別の平均
print(sample.mean()[0]) # math
print(sample.mean()[1]) # english
print(sample.mean()[2]) # science

76.0
87.25
84.0


In [14]:
# すべてのデータではないColumn別に合う値を .fillna() してあげる。
# math 
# sample['컬럼명'].fillna(sample['컬럼명'].mean()[0])
sample['math'].fillna(sample['math'].mean())

0    70.0
1    75.0
2    76.0
3    56.0
4    89.0
5    90.0
Name: math, dtype: float64

In [15]:
# english 
# sample['english'].fillna(sample.mean()[1])
sample['english'].fillna(sample['english'].mean())

0     87.25
1     65.00
2     87.25
3     89.00
4     95.00
5    100.00
Name: english, dtype: float64

In [16]:
# scienceのColumnの欠測値にscienceのColumnの平均値を入れる。
# sample['science'].fillna(sample.mean()[2])
sample['science'].fillna(sample['science'].mean())

0    84.0
1    80.0
2    84.0
3    84.0
4    83.0
5    89.0
Name: science, dtype: float64

In [17]:
tot_med = pd.Series(sample.values.reshape(18)).median()
tot_med

86.0

In [18]:
sample.fillna(tot_med)

,math,english,science
0,70.0,86.0,86.0
1,75.0,65.0,80.0
2,86.0,86.0,86.0
3,56.0,89.0,86.0
4,89.0,95.0,83.0
5,90.0,100.0,89.0


In [19]:
# median - 属性別の中位値
sample.median()[0] # math, 75
sample.median()[1] # english, 92
sample.median()[2] # science, 83

83.0

In [20]:
# すべてのデータではないColumn別に合う値を .fillna(中位値)してあげる。
# math 
# sample['math'].fillna(sample.medmedian()[0])
sample['math'].fillna(sample['math'].median())

0    70.0
1    75.0
2    75.0
3    56.0
4    89.0
5    90.0
Name: math, dtype: float64

In [21]:
# english 
sample['english'].fillna(sample['english'].median())

0     92.0
1     65.0
2     92.0
3     89.0
4     95.0
5    100.0
Name: english, dtype: float64

In [22]:
# science 
sample['science'].fillna(sample['science'].median())

0    83.0
1    80.0
2    83.0
3    83.0
4    83.0
5    89.0
Name: science, dtype: float64

In [23]:
df = pd.DataFrame({'label' : ['A', 'B', 'B', 'C', 'C', 'C', 'D']})
df

,label
0,A
1,B
2,B
3,C
4,C
5,C
6,D


In [24]:
df.describe()

,label
count,7
unique,4
top,C
freq,3


In [25]:
df.describe().loc['top']

label    C
Name: top, dtype: object

In [26]:
df['label'].value_counts()

C    3
B    2
D    1
A    1
Name: label, dtype: int64

In [27]:
df['label'].value_counts().index[0]

'C'

In [28]:
from collections import Counter

In [29]:
colors = ['red', 'blue', 'pink', 'blue', 'blue', 'red']

In [30]:
counter = Counter(colors)

In [31]:
counter

Counter({'red': 2, 'blue': 3, 'pink': 1})

In [32]:
# most_common() => 특정 요소와 그 요소의 출현 횟수를 빈출순으로 나열
counter.most_common()

[('blue', 3), ('red', 2), ('pink', 1)]

In [33]:
# 最頻値を返還するユーザ定義関数1
# 値でリストを渡せば最頻値の範疇を返還
# 関数名、媒介変数(リスト)の定義
# 関数実行文
# 1)のリストについてCounter()関数を適用
# 2)1度結果についてmost_common()関数を適用
# 3)2回結果の0番目のアイテムの0番目の要素(value、count)
# 4)3度の結果を返還/print()vs return
def mode_finder(x):
    result = Counter(x)
    mode = result.most_common()[0][0] # result.most_common() -> [('a', 2), ('b', 2), ('c', 1)]
    return mode

In [34]:
mode_finder(['a', 'a', 'b', 'b', 'c'])

'a'

In [35]:
# 同一最頻値が2個以上の場合
# for の繰り返し & if
# 同一最頻値確認 : 最頻値アイテムのvalueを探して比較
# 最終結果 : リストに同一最頻値の範疇を入れる
def mode_finder2(x):
    c = Counter(x)
    result = c.most_common()
    most_value = result[0][1]  # value가 아닌 횟수가 저장됨. 0번째 요소에 1번째 요소 : 3 = [('a', 3), ('b', 3), ('c', 3), ('d', 1)]
#   return most_value : 3
    modes = []
    
    for i in result:
        if i[1] == most_value:
            modes.append(i[0])
    return modes

In [36]:
mode_finder2(['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'c', 'd'])

['a', 'b', 'c']

In [37]:
df = pd.DataFrame({'label':['a','b','b','c','c','c','d']})
df

,label
0,a
1,b
2,b
3,c
4,c
5,c
6,d


In [38]:
Counter(df['label']).most_common()

[('c', 3), ('b', 2), ('a', 1), ('d', 1)]

### 데이터 단위 통일

#### 표준화(Standardization)
- 평균을 기준으로 얼마나 떨어져 있는지를 파악
- sklearn에서 제공하는 전처리 클래스
    - preprocessing 클래스
        - scaler() : 전체 자료의 분포를 평균 0, 표준편차 1이 되도록 스케일링
        - minmax_scale() : 최대/최소값이 각각 1, 0이 되도록 스케일링
        - StandardScaler() : scaler() 함수와 동일한 동작

- 표준화 : (요소값(하나의 데이터) - 평균) / 표준편차
- 삼성전자 vs 작은회사 주식 시세
- 몸무게 vs 키
    - 표준화 결과 : 몸무게 음수, 키 양수
    - 해석 : 몸무게는 평균 이하, 키는 평균 이상(=>마른몸)

In [39]:
# 전처리 기능을 제공하는 scikitlearn 라이브러리 및 모듈 가져오기
from sklearn.preprocessing import scale, minmax_scale

In [40]:
np.arange(9)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [41]:
# -3이상 5이하의 정수를 값으로 가지는 9행 1열 배열 생성
x = (np.arange(9) - 3).reshape(-1, 1)

In [42]:
x

array([[-3],
       [-2],
       [-1],
       [ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5]])

In [43]:
scale(x)

array([[-1.54919334],
       [-1.161895  ],
       [-0.77459667],
       [-0.38729833],
       [ 0.        ],
       [ 0.38729833],
       [ 0.77459667],
       [ 1.161895  ],
       [ 1.54919334]])

In [44]:
df = pd.DataFrame(np.hstack([x, scale(x), minmax_scale(x)]))

In [45]:
# 전체 데이터에 대한 평균, 표준편차(std : standard deviation) 비교
# x에 대한 평균, 표준편차
# scale(x)에 대한 평균, 표준편차
# minmax_scale(x)에 대한 평균, 표준편차
print('original : ', np.mean(df[0]), np.std(df[0]))
print('scale(x) : ', np.mean(df[1]), np.std(df[1]))
print('minmax_scale(x) : ', np.mean(df[2]), np.std(df[2]))

original :  1.0 2.581988897471611
scale(x) :  0.0 1.0
minmax_scale(x) :  0.5 0.3227486121839514


### 正規化(Normalizaion)

- 正規化は互いに異なる単位の多次元独立変数に対して各変数の相対的な大きさを比較する。
- 結果値は0~1の間の値を持つ。
- 単位が異なる場合
- 0がとても多い場合
- sklearnライブラリが提供するClass
    - normalize()

- 正規化 : (要素値 - 最小値) / (最大値 - 最小値)
- 全体区間を0~100に設定してデータの相対的な大きさと位置を観察する。

In [46]:
from sklearn.preprocessing import normalize

In [47]:
x = np.vstack([(np.arange(5)-20), (np.arange(5) - 2)]).T
x

array([[-20,  -2],
       [-19,  -1],
       [-18,   0],
       [-17,   1],
       [-16,   2]])

In [48]:
# 標準化(スケーリング)の適用
y1 = scale(x)

In [49]:
# 正規化適用
y2 = normalize(x)

In [50]:
# 正規化有無の確認
# np.linalg.norm(data、axis=1):値(np.array、ベクトル)の単位

In [51]:
print(x)
print('x norm', np.linalg.norm(x, axis=1))

[[-20  -2]
 [-19  -1]
 [-18   0]
 [-17   1]
 [-16   2]]
x norm [20.09975124 19.02629759 18.         17.02938637 16.1245155 ]


In [52]:
print(y1)
print('scale norm', np.linalg.norm(y1, axis=1))

[[-1.41421356 -1.41421356]
 [-0.70710678 -0.70710678]
 [ 0.          0.        ]
 [ 0.70710678  0.70710678]
 [ 1.41421356  1.41421356]]
scale norm [2. 1. 0. 1. 2.]


In [53]:
print(y2)
print('normalize norm', np.linalg.norm(y2, axis=1))

[[-0.99503719 -0.09950372]
 [-0.99861783 -0.05255883]
 [-1.          0.        ]
 [-0.99827437  0.05872202]
 [-0.99227788  0.12403473]]
normalize norm [1. 1. 1. 1. 1.]
